In [1]:
%load_ext autoreload
%autoreload 2


TODO

S3 working with creds

[EFS on clusters maybe](https://docs.ray.io/en/latest/cluster/vms/user-guides/launching-clusters/aws.html#start-ray-with-the-ray-cluster-launcher)

In [2]:
import time

import ray

import celltrip


In [ ]:
# import s3fs
# s3 = s3fs.S3FileSystem()
# s3.open('s3://nkalafut-celltrip/test', 'w').close()

NoCredentialsError: Unable to locate credentials

In [3]:
# Arguments
import argparse
parser = argparse.ArgumentParser(description='Train CellTRIP model', formatter_class=argparse.ArgumentDefaultsHelpFormatter)

# Important parameters
# group = parser.add_argument_group('Input Data')
parser.add_argument('input_files', type=str, nargs='*', help='h5ad files to be used for input')
parser.add_argument('--merge_files', type=str, action='append', nargs='+', help='h5ad files to merge as input')
parser.add_argument('--partition_cols', type=str, action='append', nargs='+', help='Columns for data partitioning, found in `adata.obs` DataFrame')
parser.add_argument('--data_in_memory', action='store_true', help='Load data into memory for sampling')
parser.add_argument('--logfile', type=str, default='cli', help='Location for log file, can be `cli`, `<local_file>`, or `<s3 location>`')

# Notebook defaults and script handling
if not celltrip.utility.notebook.is_notebook():
    config = parser.parse_args()
else:
    config = parser.parse_args((
        './data/MERFISH/expression.h5ad ./data/MERFISH/spatial.h5ad '
        '--logfile s3://nkalafut-celltrip/log.txt'
        ).split(' '))

print(config)


Namespace(input_files=['./data/MERFISH/expression.h5ad', './data/MERFISH/spatial.h5ad'], merge_files=None, partition_cols=None, data_in_memory=False, logfile='s3://nkalafut-celltrip/log.txt')


In [ ]:
ray.shutdown()
ray.init(
    address='ray://100.85.187.118:10001',
    runtime_env={
        'py_modules': [celltrip],
        'pip': '../requirements.txt',
        'env_vars': {
            'RAY_DEDUP_LOGS': '0',
            # 'NCCL_SOCKET_IFNAME': 'tailscale',  # lo,en,wls,docker,tailscale
        }})


2025-04-06 20:19:27,425	INFO client_builder.py:244 -- Passing the following kwargs to ray.init() on the server: log_to_driver
SIGTERM handler is not set because current thread is not the main thread.


Python version:,3.10.16
Ray version:,2.43.0
Dashboard:,http://100.85.187.118:8265


(raylet) [2025-04-06 20:19:38,919 E 1339472 1339507] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2025-04-06_19-14-03_799214_1339203 is over 95% full, available space: 23.8669 GB; capacity: 929.203 GB. Object creation will fail if spilling is required.
(raylet) [2025-04-06 20:19:48,930 E 1339472 1339507] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2025-04-06_19-14-03_799214_1339203 is over 95% full, available space: 23.8669 GB; capacity: 929.203 GB. Object creation will fail if spilling is required.
(raylet) [2025-04-06 20:19:58,938 E 1339472 1339507] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2025-04-06_19-14-03_799214_1339203 is over 95% full, available space: 23.8668 GB; capacity: 929.203 GB. Object creation will fail if spilling is required.
(raylet) [2025-04-06 20:20:08,948 E 1339472 1339507] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2025-04-06_19-14-03_799214_1339203 is over 95% full, available space: 23.8668 GB; capacity: 929.203 GB. Obj

In [5]:
@ray.remote(num_cpus=1e-4)
def train(config):
    import celltrip

    # Initialization
    def env_init():
        # Create dataloader
        adatas = celltrip.utility.processing.read_adatas(*config.input_files, on_disk=(not config.data_in_memory))
        if config.merge_files is not None:
            for merge_files in config.merge_files:
                merge_adatas = celltrip.utility.processing.read_adatas(*merge_files, on_disk=(not config.data_in_memory))
                adatas += celltrip.utility.processing.merge_adatas(*merge_adatas, on_disk=(not config.data_in_memory))
        celltrip.utility.processing.test_adatas(*adatas, partition_cols=config.partition_cols)
        dataloader = celltrip.utility.processing.PreprocessFromAnnData(
            *adatas, partition_cols=config.partition_cols,  num_nodes=200,
            pca_dim=128, seed=42)
        # modalities, adata_obs, adata_vars = dataloader.sample()
        # Return env
        return celltrip.environment.EnvironmentBase(
            dataloader, dim=3, penalty_bound=1)

    # Default 25Gb Forward, 14Gb Update, at max capacity
    policy_init = lambda env: celltrip.policy.PPO(
        2*env.dim, env.dataloader.modal_dims, env.dim)  # update_iterations=2, minibatch_size=3e3,

    memory_init = lambda policy: celltrip.memory.AdvancedMemoryBuffer(
        sum(policy.modal_dims), split_args=policy.split_args)  # prune=100

    initializers = (env_init, policy_init, memory_init)

    stage_functions = [
        # lambda w: w.env.set_rewards(penalty_velocity=1, penalty_action=1),
        # lambda w: w.env.set_rewards(reward_origin=1),
        # lambda w: w.env.set_rewards(reward_origin=0, reward_distance=1),
    ]

    # Run function
    # rollout_kwargs={'dummy': True}, update_kwargs={'update_iterations': 5}, sync_across_nodes=False
    celltrip.train.train_celltrip(
        initializers=initializers,
        num_gpus=2,
        num_learners=2,
        num_runners=4,
        stage_functions=stage_functions,
        logfile=config.logfile)

ray.get(train.remote(config))


KeyboardInterrupt: 

In [ ]:
# env = env_init(parent_dir=True).to('cuda')
# policy = policy_init(env).to('cuda')
# memory = memory_init(policy)
# celltrip.train.simulate_until_completion(env, policy, memory)
# memory.propagate_rewards()
# memory.normalize_rewards()
# # memory.fast_sample(10_000, shuffle=False)
# len(memory)

# memory.mark_sampled()
# env.reset()
# celltrip.train.simulate_until_completion(env, policy, memory)
# memory.propagate_rewards()
# memory.adjust_rewards()
